In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("./new_preprocessed_final.csv")

In [8]:
# Select TF‑IDF + token overlap features
feature_cols = [c for c in df.columns if ("cosine" in c.lower() and "bert" not in c.lower()) or "overlap" in c.lower()]

# Label
label_col = None
for c in df.columns:
  if "is_clickbait" in c.lower():
    label_col = c
    break
if label_col is None:
  raise ValueError("Kolom label tidak ditemukan. Pastikan ada kolom is_clickbait*.")

X = df[feature_cols]
y = df[label_col]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42, stratify=y
)

# Perceptron (Single-Layer) with scaling
pipe = Pipeline([
  ("scaler", StandardScaler()),
  ("clf", Perceptron(max_iter=1000, tol=1e-3, class_weight="balanced", random_state=42))
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("== Features used ==")
print(feature_cols)
print("\n=== Classification Report (Perceptron + TF-IDF + Token Overlap) ===")
print(classification_report(y_test, y_pred))
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

== Features used ==
['cosine_similarity', 'token_overlap']

=== Classification Report (Perceptron + TF-IDF + Token Overlap) ===
              precision    recall  f1-score   support

           0       1.00      0.92      0.96       514
           1       0.67      0.98      0.79        86

    accuracy                           0.93       600
   macro avg       0.83      0.95      0.87       600
weighted avg       0.95      0.93      0.93       600


=== Confusion Matrix ===
[[472  42]
 [  2  84]]


In [9]:
# TF‑IDF + token overlap + extra features
tfidf_feats = [c for c in df.columns if ("cosine" in c.lower() and "bert" not in c.lower()) or "overlap" in c.lower()]

# Extra linguistic features
extra_feats = ["title_length","sensational_word_count","punctuation_count","punctuation_ratio"]
extra_feats = [c for c in extra_feats if c in df.columns]

# Combine
feature_cols = tfidf_feats + extra_feats

# Label
label_col = None
for c in df.columns:
  if "is_clickbait" in c.lower():
    label_col = c
    break
if label_col is None:
  raise ValueError("Kolom label tidak ditemukan. Pastikan ada kolom is_clickbait*.")

X = df[feature_cols]
y = df[label_col]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42, stratify=y
)

# Perceptron (Single-Layer) with scaling
pipe = Pipeline([
  ("scaler", StandardScaler()),
  ("clf", Perceptron(max_iter=1000, tol=1e-3, class_weight="balanced", random_state=42))
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("== Features used ==")
print(feature_cols)
print("\n=== Classification Report (Perceptron + TF-IDF + Token Overlap) ===")
print(classification_report(y_test, y_pred))
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

== Features used ==
['cosine_similarity', 'token_overlap', 'title_length', 'sensational_word_count', 'punctuation_count', 'punctuation_ratio']

=== Classification Report (Perceptron + TF-IDF + Token Overlap) ===
              precision    recall  f1-score   support

           0       0.99      0.90      0.94       514
           1       0.61      0.94      0.74        86

    accuracy                           0.91       600
   macro avg       0.80      0.92      0.84       600
weighted avg       0.93      0.91      0.91       600


=== Confusion Matrix ===
[[462  52]
 [  5  81]]
